In [1]:
import os
import sys
import os

current_dir = os.path.abspath(os.path.dirname(os.path.realpath("__file__")))
crepe_path = os.path.join(current_dir, '..', 'crepe')
sys.path.append(crepe_path)

import crepe
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Using Crepe to Extract Frequency audio features

In [ ]:
def pitch_detection(input_audio_path):
    # Load the pre-trained crepe model
    model_capacity = 'small'

    # Load the guitar audio file
    y, sr = librosa.load(input_audio_path)

    # Pitch detection
    time, frequency, confidence, activation = crepe.predict(y, sr, viterbi=True, model_capacity=model_capacity)

    # Plot the pitch detection results
    #plt.figure(figsize=(10, 6))
    #plt.subplot(3, 1, 1)
    #plt.plot(time, frequency, 'r')
    # plt.title('Estimated Pitch (CREPE)')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Frequency (Hz)')

    # plt.subplot(3, 1, 2)
    # plt.plot(time, confidence, 'g')
    # plt.title('Pitch Confidence')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Confidence')

    # plt.subplot(3, 1, 3)
    # plt.plot(time, activation, 'b')
    # plt.title('Activation')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Activation')

    # plt.tight_layout()
    # plt.show()


    return np.array(time), np.array(frequency)


In [ ]:
relative_input_path = "input-audios/guitar.wav"
input_path = os.path.abspath(os.path.join(os.getcwd(), relative_input_path))
time_values, frequency_values = pitch_detection(input_path)

In [ ]:
# y, sr = librosa.load(input_path)

# # Compute the Short-Time Fourier Transform (STFT)
# hop_length = 512  #adustable
# n_fft = 2048  # adustable
# stft = librosa.core.stft(y, hop_length=hop_length, n_fft=n_fft)

# # Convert amplitude to decibels
# stft_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

# # Extract time and frequency
# time_values = librosa.times_like(stft)
# frequency_values = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

In [ ]:
import scipy.io.wavfile as wav

def interpolate_linearly(wave_table, index):
    truncated_index = int(np.floor(index))
    next_index = (truncated_index + 1) % wave_table.shape[0]
    next_index_weight = index - truncated_index
    truncated_index_weight = 1 - next_index_weight
    return truncated_index_weight * wave_table[truncated_index] + next_index_weight * wave_table[next_index]

def fade_in_out(signal, fade_length=1000):
    if fade_length > len(signal):
        fade_length = len(signal)

    fade_in = (1 - np.cos(np.linspace(0, np.pi, fade_length))) * 0.5
    fade_out = np.flip(fade_in)

    signal[:fade_length] = np.multiply(fade_in, signal[:fade_length])

    if len(signal) > fade_length:
        signal[-fade_length:] = np.multiply(fade_out, signal[-fade_length:])

    return signal

def generate_wave_table(wavetable_length=64):
    wave_table = np.zeros((wavetable_length,))
    for n in range(wavetable_length):
        wave_table[n] = np.sin(2 * np.pi * n / wavetable_length)
    return wave_table

def wavetable_synthesis(time, frequency, sample_rate=44100, wavetable_length=64, fade_length=1000):
    output = np.zeros((0,))
    wave_table = generate_wave_table(wavetable_length)

    for t, f in zip(time, frequency):
        current_signal = np.zeros((int(t * sample_rate),))
        index = 0
        index_increment = f * wavetable_length / sample_rate

        for n in range(current_signal.shape[0]):
            current_signal[n] = interpolate_linearly(wave_table, index)
            index += index_increment
            index %= wavetable_length

        gain = -20
        amplitude = 10 ** (gain / 20)
        current_signal *= amplitude

        current_signal = fade_in_out(current_signal, fade_length)

        # Concatenate the current signal to the output
        output = np.concatenate((output, current_signal))

    return output

def main():
    sample_rate = 44100

    time = time_values
    frequency = frequency_values 
    synthesized_output = wavetable_synthesis(time, frequency, sample_rate)

    relative_output_path = "output-audios/synthesised.wav"
    output_path = os.path.abspath(os.path.join(os.getcwd(), relative_output_path))


    # Save the final synthesized output
    wav.write(output_path, sample_rate, synthesized_output.astype(np.float32))

if __name__ == "__main__":
    main()


In [ ]:
# import numpy as np
# import sounddevice as sd

# def generate_piano_sound(time_data, frequency_data, sample_rate=44100):
#     time = np.arange(0, max(time_data), 1/sample_rate)

#     # Interpolate frequencies for the given time
#     frequency = np.interp(time, time_data, frequency_data)

#     # Generate piano sound using sine waves
#     piano_sound = np.zeros_like(time)
#     for freq in frequency:
#         piano_sound += np.sin(2 * np.pi * freq * time)

#     # Normalize the sound to prevent clipping
#     piano_sound /= np.max(np.abs(piano_sound))

#     return piano_sound

In [ ]:
# from scipy.io import wavfile

# def create_piano_from_guitar(input_guitar_path, output_audio_path):
#     # Pitch detection
#     time_data, estimated_pitch = pitch_detection(input_guitar_path)

#     # Generate piano sound based on time and frequency data
#     piano_sound = generate_piano_sound(time_data, estimated_pitch)

#     print("piano_sound")

#     sd.play(piano_sound, samplerate=44100)
#     sd.wait()


#     # # Save the generated piano sound to a file (optional
#     # wavfile.write(output_audio_path, 44100, np.int16(piano_sound * 32767))

# # Example usage
# input_guitar_path = "/input-audios/guitar.wav"
# output_audio_path = "/output_audios/generated.wav"

# create_piano_from_guitar(input_guitar_path, output_audio_path)

In [ ]:
# def generate_wave_table(frequency, duration=1.0, sample_rate=44100):
#     time = np.arange(0, duration, 1/sample_rate)

#     # Generate sine waveforms for each frequency
#     wave_table = np.sin(2 * np.pi * frequency * time[:, np.newaxis])

#     return wave_table

# def wave_table_synthesis(input_audio_path):
#     # Pitch detection using Crepe
#     time, frequency = pitch_detection(input_audio_path)

#     # Generate wave table
#     wave_table = generate_wave_table(frequency)

#     # Synthesize audio using wave table
#     synthesized_audio = np.sum(wave_table, axis=1)

#     # Play the synthesized audio using Sounddevice
#     sd.play(synthesized_audio, samplerate=44100)
#     sd.wait()
# input_audio_path = "/input-audios/guitar.wav"
# wave_table_synthesis(input_audio_path)